In [55]:
import nltk
import json
import pandas as pd
from glob import glob
from konlpy.tag import Okt
from nltk.tokenize import word_tokenize
twitter = Okt()

In [56]:
caption_list=[]

files = glob('../data/museum/*/*.json')
for filename in files:
    with open(filename, encoding='UTF8') as f:
        d = json.load(f)
        for key in d.keys():
            for i in range(len(d[str(key)])):
                caption_list.append(d[str(key)][i]['caption'])

with open("../data/museum/caption.txt","w", encoding = 'UTF8') as f:
    for caption in caption_list:
        f.write(f'{caption}\n')

In [57]:
with open("../data/museum/caption.txt", encoding='UTF8') as f:
    captions=f.readlines()

In [58]:
nltk.download('punkt')
stop_words = ['.','있다.','있는','이','의','를','에','을', '있', '하', '것', '들', '그', '되', 
              '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', 
              '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하']

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cho40\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [59]:
caption = pd.DataFrame({'caption':captions})
caption['clean_caption'] = caption['caption'].str.replace("^[가-힣a-zA-Z]+$", " ")

<ipython-input-59-da9e2d61ff69>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  caption['clean_caption'] = caption['caption'].str.replace("^[가-힣a-zA-Z]+$", " ")


In [60]:
caption['clean_caption'][0]
tokenized_caption = caption['clean_caption'].apply(lambda x: x.split()) # 토큰화
tokenized_caption = tokenized_caption.apply(lambda x: [item for item in x if item not in stop_words])
result = []

In [61]:
from gensim import corpora

In [62]:
dictionary = corpora.Dictionary(tokenized_caption)
corpus = [dictionary.doc2bow(text) for text in tokenized_caption]

In [63]:
import gensim
NUM_TOPICS = 10 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.194*"앉아" + 0.056*"여자가" + 0.039*"벤치에" + 0.038*"의자에" + 0.033*"테이블에" + 0.027*"창문에" + 0.021*"남자가" + 0.017*"거울에" + 0.017*"비친" + 0.016*"여자"')
(1, '0.101*"큰" + 0.058*"옆에" + 0.057*"창문" + 0.043*"건물" + 0.030*"간판" + 0.029*"앉아" + 0.025*"창문이" + 0.022*"아이가" + 0.022*"앞에" + 0.019*"아래에"')
(2, '0.109*"무리의" + 0.094*"사람들이" + 0.091*"서" + 0.067*"건물" + 0.036*"밖에" + 0.033*"남자" + 0.030*"건물의" + 0.024*"옆에" + 0.019*"측면에" + 0.018*"큰"')
(3, '0.091*"나무" + 0.066*"앞에" + 0.052*"옆에" + 0.047*"가게" + 0.044*"서" + 0.044*"금속" + 0.028*"텔레비전" + 0.026*"텔레비전이" + 0.020*"건물" + 0.020*"된"')
(4, '0.170*"위에" + 0.091*"놓여" + 0.078*"탁자" + 0.072*"나무" + 0.028*"앉아" + 0.020*"테이블" + 0.017*"책상" + 0.016*"컴퓨터" + 0.015*"많은" + 0.012*"하얀"')
(5, '0.096*"어린" + 0.050*"들고" + 0.046*"소년이" + 0.038*"소녀가" + 0.035*"사진" + 0.024*"셔츠를" + 0.024*"넥타이를" + 0.024*"입은" + 0.023*"서" + 0.022*"여자가"')
(6, '0.177*"벽에" + 0.067*"걸린" + 0.061*"걸려" + 0.043*"그림" + 0.036*"그려진" + 0.028*"그림이" + 0.021*"그림의" + 0.020*"매고" + 0.017*"기대어" + 0.015*"붙어"')
(7, '0.090*"색" + 0.057*"빨간" 

In [64]:
vector = ldamodel[test_doc]
print(vector)

[(0, 0.41999856), (1, 0.020000657), (2, 0.020000083), (3, 0.020000102), (4, 0.020000648), (5, 0.41999966), (6, 0.020000083), (7, 0.020000083), (8, 0.020000083), (9, 0.020000083)]


In [65]:
test_doc = corpus[-1]
vector = ldamodel[test_doc]
ldamodel.save('../data/museum/model_topic/test_model.model')

In [66]:
new_model = gensim.models.ldamodel.LdaModel.load("../data/museum/model_topic/test_model.model")
